In [17]:
import pandas as pd
import numpy as np
import pyarrow
import tarfile
from matplotlib import pyplot as plt

with tarfile.open('mfg_eu_temps_20191003.tar.gz', mode='r:gz') as tf:
    tf.extractall('C:Users/zhang/Box/Python/SIPH')
    mfg_eu_temps = pd.read_parquet('C:Users/zhang/Box/Python/SIPH/mfg_eu_temps_20191003/', engine='pyarrow') 

In [18]:
fraction=pd.read_csv('WHP fraction.csv')
whp_frac = fraction.loc[:,('naics','end use','Overall_fraction')]
COP=pd.read_csv('COP.csv')
COP_v = COP.loc[:,('naics','end use','COP')]

In [19]:
#fraction with different end uses
whp_frac_bchp=whp_frac[whp_frac['end use'] == "Boiler/CHP"]
whp_frac_ph=whp_frac[whp_frac['end use']=="Process heating"]
COP_v_bchp=COP_v[COP_v['end use']=="Boiler/CHP"]
COP_v_ph=COP_v[COP_v['end use']=="Process heating"]

In [20]:
#filter for different end uses
boiler_chp=mfg_eu_temps.loc[(mfg_eu_temps['End_use']=='Conventional Boiler Use')|(mfg_eu_temps['End_use']=="CHP and/or Cogeneration Process")].copy()
ph=mfg_eu_temps.loc[(mfg_eu_temps['End_use']=='Process Heating')].copy() 

In [21]:
#group data by county, fuel types, and naics code
boiler_chp_group = boiler_chp.groupby(['MECS_FT','COUNTY_FIPS','naics','End_use']).sum().reset_index()
ph_group=ph.groupby(['MECS_FT','COUNTY_FIPS','naics','End_use']).sum().reset_index()

In [22]:
#to calculate the technical potential which naics code is with 6 digits
percentage_bchp = pd.merge(whp_frac_bchp, COP_v_bchp, how='inner', on=['naics'])
percentage_bchp['overall']=percentage_bchp.Overall_fraction * percentage_bchp.COP
tech_pot_bchp = pd.merge(percentage_bchp,boiler_chp_group, how='inner', on=['naics'])
tech_pot_bchp['tech_potential'] = tech_pot_bchp.MMBtu * tech_pot_bchp.overall
tech_pot_bchpn = tech_pot_bchp[['naics','end use_x','MECS_FT','COUNTY_FIPS','tech_potential']]
percentage_ph = pd.merge(whp_frac_ph, COP_v_ph, how='inner', on=['naics'])
percentage_ph['overall'] = percentage_ph.Overall_fraction * percentage_ph.COP
tech_pot_ph = pd.merge(percentage_ph, ph_group, how = 'inner', on=['naics'])
tech_pot_ph['tech_potential'] = tech_pot_ph.MMBtu * tech_pot_ph.overall
tech_pot_phn = tech_pot_ph[['naics','end use_x','MECS_FT','COUNTY_FIPS','tech_potential']]

In [23]:
#save to file
tech_pot_bchpn.to_csv(r'C:\Users\zhang\Box\Python\SIPH\tech_potential_bchp_1.csv', index = False)
tech_pot_phn.to_csv(r'C:\Users\zhang\Box\Python\SIPH\tech_potential_ph_1.csv', index = False)

In [32]:
#calculate the technical potential with 3-digit naics code
food = boiler_chp_group[(boiler_chp_group.naics<312000) & (boiler_chp_group.naics>311000)]
food_fraction = whp_frac_bchp[whp_frac_bchp.naics==311]
food_cop = COP_v_bchp[COP_v_bchp.naics==311]
food.loc[:,'recovered_heat'] = food.MMBtu * food_fraction.Overall_fraction.iloc[0]
food.loc[:,'tech_potential'] = food.recovered_heat * food_cop.COP.iloc[0]
food_pot = food[['naics','End_use','MECS_FT','COUNTY_FIPS','tech_potential']]

bev = boiler_chp_group[(boiler_chp_group.naics<313000) & (boiler_chp_group.naics>312000)]
bev_fraction = whp_frac_bchp[whp_frac_bchp.naics==312]
bev_cop = COP_v_bchp[COP_v_bchp.naics==312]
bev.loc[:,'recovered_heat'] = bev.MMBtu * bev_fraction.Overall_fraction.iloc[0]
bev.loc[:,'tech_potential'] = bev.recovered_heat * bev_cop.COP.iloc[0]
bev_pot = bev[['naics','End_use','MECS_FT','COUNTY_FIPS','tech_potential']]

textile = boiler_chp_group[(boiler_chp_group.naics<314000) & (boiler_chp_group.naics>313000)]
textile_fraction = whp_frac_bchp[whp_frac_bchp.naics==313]
textile_cop = COP_v_bchp[COP_v_bchp.naics==313]
textile.loc[:,'recovered_heat']=textile.MMBtu * textile_fraction.Overall_fraction.iloc[0]
textile.loc[:,'tech_potential'] = textile.recovered_heat * textile_cop.COP.iloc[0]
text_pot = textile[['naics','End_use','MECS_FT','COUNTY_FIPS','tech_potential']]

wood = boiler_chp_group[(boiler_chp_group.naics<322000) & (boiler_chp_group.naics>321000)]
wood_fraction = whp_frac_bchp[whp_frac_bchp.naics==321]
wood_cop = COP_v_bchp[COP_v_bchp.naics==321]
wood.loc[:,'recovered_heat']=wood.MMBtu*wood_fraction.Overall_fraction.iloc[0]
wood.loc[:,'tech_potential']=wood.MMBtu * wood_fraction.Overall_fraction.iloc[0]
wood_pot = wood[['naics','End_use','MECS_FT','COUNTY_FIPS','tech_potential']]

paper = boiler_chp_group[(boiler_chp_group.naics<323000) & (boiler_chp_group.naics>322000)]
paper_fraction = whp_frac_bchp[whp_frac_bchp.naics==322]
paper_cop = COP_v_bchp[COP_v_bchp.naics==322]
paper.loc[:,'recovered_heat']=paper.MMBtu*paper_fraction.Overall_fraction.iloc[0]
paper.loc[:,'tech_potential']=paper.recovered_heat * paper_cop.COP.iloc[0]
paper_pot = paper[['naics','End_use','MECS_FT','COUNTY_FIPS','tech_potential']]

petroleum_bch = boiler_chp_group[(boiler_chp_group.naics<325000) & (boiler_chp_group.naics>324000)]
petroleum_ph = ph_group[(ph_group.naics<325000) & (ph_group.naics>324000)]
petroleum_bch_fraction=whp_frac_bchp[whp_frac_bchp.naics==324]
petroleum_ph_fraction=whp_frac_ph[whp_frac_ph.naics==324]
petroleum_cop_bch = COP_v_bchp[COP_v_bchp.naics==324]
petroleum_cop_ph = COP_v_bchp[COP_v_bchp.naics==324]
petroleum_bch.loc[:,'distillation_recovered'] = petroleum_bch.MMBtu * petroleum_bch_fraction.Overall_fraction.iloc[0]
petroleum_bch.loc[:,'general_recovered'] = petroleum_bch.MMBtu * petroleum_bch_fraction.Overall_fraction.iloc[1]
petroleum_ph.loc[:,'ph_recovered']=petroleum_ph.MMBtu * petroleum_ph_fraction.Overall_fraction.iloc[0]
petroleum_bch.loc[:,'distillation_potential'] = petroleum_bch.distillation_recovered * petroleum_cop_bch.COP.iloc[0]
petroleum_bch.loc[:,'general_potential'] = petroleum_bch.general_recovered * petroleum_cop_bch.COP.iloc[1]
petroleum_ph.loc[:,'tech_potential'] = petroleum_ph.ph_recovered * petroleum_cop_ph.COP.iloc[0]
petro_bch_dis_pot = petroleum_bch[['naics','End_use','MECS_FT','COUNTY_FIPS','distillation_potential']]
petro_bch_gen_pot = petroleum_bch[['naics','End_use','MECS_FT','COUNTY_FIPS','general_potential']]
petro_ph_pot = petroleum_ph[['naics','End_use','MECS_FT','COUNTY_FIPS','tech_potential']]

chemical = boiler_chp_group[(boiler_chp_group.naics<326000) & (boiler_chp_group.naics>325000)]
chemical_fraction = whp_frac_bchp[whp_frac_bchp.naics==325]
chemical_cop = COP_v_bchp[COP_v_bchp.naics==325]
chemical.loc[:,'recovered_heat']=chemical.MMBtu*chemical_fraction.Overall_fraction.iloc[0]
chemical.loc[:,'tech_potential']=chemical.recovered_heat * chemical_cop.COP.iloc[0]
chemical_pot = chemical[['naics','End_use','MECS_FT','COUNTY_FIPS','tech_potential']]

C:\Users\zhang\anaconda3\lib\site-packages\pandas\core\indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\zhang\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [36]:
new = [food_pot, bev_pot]
result = pd.contat(new)
result

AttributeError: module 'pandas' has no attribute 'contat'

In [33]:
#save to file
food_pot.to_csv(r'C:\Users\zhang\Box\Python\SIPH\food_pot_1.csv', index = False)
bev_pot.to_csv(r'C:\Users\zhang\Box\Python\SIPH\bev_pot.csv', index = False)
text_pot.to_csv(r'C:\Users\zhang\Box\Python\SIPH\text_pot.csv', index = False)
wood_pot.to_csv(r'C:\Users\zhang\Box\Python\SIPH\wood_pot.csv', index = False)
paper_pot.to_csv(r'C:\Users\zhang\Box\Python\SIPH\paper_pot.csv', index = False)
petro_bch_dis_pot.to_csv(r'C:\Users\zhang\Box\Python\SIPH\petro_bch_dis_pot.csv', index = False)
petro_bch_gen_pot.to_csv(r'C:\Users\zhang\Box\Python\SIPH\petro_bch_gen_pot.csv', index = False)
petro_ph_pot.to_csv(r'C:\Users\zhang\Box\Python\SIPH\petro_ph_pot.csv', index = False)
chemical_pot.to_csv(r'C:\Users\zhang\Box\Python\SIPH\chemical_pot.csv', index = False)

In [37]:
# combined saved files together mannually
tech_pot_overall=pd.read_csv('Tech_potential_overall.csv')
tech_pot_overall.head()

,naics,End_use,MECS_FT,COUNTY_FIPS,tech_potential
0,311314,CHP and/or Cogeneration Process,Coal,1003,6.899464
1,311314,CHP and/or Cogeneration Process,Coal,1057,6.899464
2,311314,CHP and/or Cogeneration Process,Coal,1129,77.722483
3,311612,CHP and/or Cogeneration Process,Coal,2020,0.162352
4,311611,CHP and/or Cogeneration Process,Coal,2050,0.162352


In [41]:
#group the results by county FIPS, naics code, fuel type, and end uses
tech_pot_overall_group=tech_pot_overall.groupby(['naics','End_use','MECS_FT','COUNTY_FIPS']).sum().reset_index()
tech_pot_overall_group.head()

,naics,End_use,MECS_FT,COUNTY_FIPS,tech_potential
0,311111,Conventional Boiler Use,Natural_gas,19163,11097.767250
1,311111,Conventional Boiler Use,Natural_gas,39049,9663.467808
2,311119,CHP and/or Cogeneration Process,Diesel,24045,1.260384
3,311119,CHP and/or Cogeneration Process,Natural_gas,19179,17138.619840
4,311119,CHP and/or Cogeneration Process,Natural_gas,24045,5324.850000


In [42]:
#save this into file
tech_pot_overall_group.to_csv(r'C:\Users\zhang\Box\Python\SIPH\tech_pot_overall_group.csv', index = False)